In [ ]:
import os
import tensorflow as tf
import cv2
import numpy as np
import pickle
import random
import torch
from ultralytics import YOLO
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.callbacks import EarlyStopping

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
session = InteractiveSession(config=config)

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
# Load YOLOv5 model for face detection
yolo_face = YOLO("yolov5s.pt")

In [ ]:
# Define categories
categories = ['with_mask', 'without_mask']

In [ ]:
# Data Augmentation
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255
)

In [ ]:
# Load and preprocess dataset
data = []
for category in categories:
    path = os.path.join('train', category)
    label = categories.index(category)  # 0 for with_mask, 1 for without_mask

    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        data.append([img, label])

In [ ]:
# Shuffle data to mix classes
random.shuffle(data)

In [ ]:
# Prepare input and labels
X = []
y = []
for features, label in data:
    X.append(features)
    y.append(label)

In [ ]:
X = np.array(X) / 255.0  # Normalize images
y = np.array(y)

In [ ]:
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Apply augmentation only to training set
train_generator = data_gen.flow(X_train, y_train, batch_size=32)

In [ ]:
# Build CNN model
model = Sequential([
    Flatten(input_shape=(224, 224, 3)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)

In [ ]:
# Train model with augmentation
model.fit(train_dataset, epochs=20, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
pickle.dump(model,open('yolo_model.pkl','wb'))

In [ ]:
# Function to predict mask status
def predict_face_mask(img):
    img = img / 255.0  # Normalize
    y_pred = model.predict(img.reshape(1, 224, 224, 3))
    return "Without Mask" if y_pred[0][0] > 0.5 else "With Mask"

In [ ]:
# Load test images
sample1 = cv2.imread('mask.jpg')
sample1 = cv2.resize(sample1, (224, 224))

sample2 = cv2.imread('without_mask.jpg')
sample2 = cv2.resize(sample2, (224, 224))

In [ ]:
print("Sample1 Prediction:", predict_face_mask(sample1))
print("Sample2 Prediction:", predict_face_mask(sample2))

In [ ]:
# Face detection with YOLO
def detect_faces(img):
    results = yolo_face(img)  # Run YOLO detection
    faces = []
    for result in results:
        for box in result.boxes.xyxy:  # Get bounding boxes
            x, y, x2, y2 = map(int, box)
            faces.append((x, y, x2 - x, y2 - y))  # Convert to (x, y, w, h)
    return faces

In [ ]:
# Draw label function
def draw_label(img, text, pos, bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)[0]
    end_x, end_y = pos[0] + text_size[0] + 2, pos[1] - text_size[1] - 2
    cv2.rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 1, cv2.LINE_AA)

In [ ]:
# Live webcam detection
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    faces = detect_faces(frame)
    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224)) / 255.0  # Normalize
        prediction = predict_face_mask(face)

        color = (0, 255, 0) if prediction == "With Mask" else (0, 0, 255)
        draw_label(frame, prediction, (x, y-10), color)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

    cv2.imshow("Mask Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()